Uso de librerias correspondientes para el desarrollo del proyecto

In [29]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split#60,20,20
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
(X,Y),(X1,Y1)=tf.keras.datasets.mnist.load_data()

In [30]:
X.shape

(60000, 28, 28)

Se observa una dimensión de 28x28 pero se ajustará a 32x32 como en el articulo

In [31]:
x_conv_t=np.pad(X,pad_width=(((0,0),(2,2),(2,2))))
x_conv_p=np.pad(X1,pad_width=((0,0),(2,2),(2,2)))
print(x_conv_t.shape)
print(x_conv_p.shape)
# Se debe agregar un 1 al final para agregar el unico canal
x_conv_t=x_conv_t.reshape(x_conv_t.shape[0],32,32,1)
x_conv_p=x_conv_p.reshape(x_conv_p.shape[0],32,32,1)

(60000, 32, 32)
(10000, 32, 32)


Se realizará una normalización de los valores al intervalo de $[0,1]$ 
$$x_{norm}[a,b]=(b-a)\frac{x-x_{min}}{x_{max}-x_{min}}+a$$

para posteriormente, realizar la configuración de ONE HOT. 

In [32]:
x_conv_t=(x_conv_t/x_conv_t.max())
x_conv_p=(x_conv_p/x_conv_p.max())
print(x_conv_t.max(),x_conv_p.max())
## ONE  Hot
from tensorflow.keras.utils import to_categorical
print(Y[3450])

yt_oh=to_categorical(Y,Y.max()+1)
yp_oh=to_categorical(Y1,Y1.max()+1)
print(yt_oh[3450])
print(Y1[200])
print(yp_oh[200])
#### Verificando dimensiones
print(x_conv_t.shape, yt_oh.shape)
print(x_conv_p.shape,yp_oh.shape)

1.0 1.0
4
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
3
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
(60000, 32, 32, 1) (60000, 10)
(10000, 32, 32, 1) (10000, 10)


Una vez lista las las normalizaciones adecuadas, se plantea la estructura de la red convolucional.

## Empezando por **C1**

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model=Sequential()
model.add(tf.keras.layers.Conv2D(6,(5,5),input_shape=(32,32,1),activation='tanh',padding='valid',strides=1)) #C1


# **S2** 

In [34]:
model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2))) #S2

# C3

In [35]:
model.add(tf.keras.layers.Conv2D(16,(5,5),activation='tanh',padding='valid',strides=1)) #c3

# S4

In [36]:
model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2))) #s4
model.add(tf.keras.layers.Flatten())

# C5,C6

In [37]:
model.add(Dense(120,activation='tanh')) #c5
model.add(Dense(84,activation='tanh')) #c6


Para el caso de **C7** que es la salida, se comenta que la estructura de la red es de forma RBF, el cual tiene la siguiente estructura:

$$y_i=\sum_j (x_j-\omega_{ij})^2 $$

Se encontró un código que permite caracterizar este tipo de neurona.

Obtenido de: https://stackoverflow.com/questions/53855941/how-to-implement-rbf-activation-function-in-keras


In [38]:
from keras.layers import Layer
from keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

# Agregando C7

In [39]:
model.add(RBFLayer(10,0.5)) #c7
#model.add(Dense(10,activation='softmax')) #c7

Posteriormente se compila el modelo con las funciones de perdida y el optimizador, el cual nos indica el articulo que es: 

Función de perdida (similitud con la entropía cruzada)
$$ E(W)=\frac{1}{P}\sum_{p=1}^P (yD^p(Z^p,W)+log(e^{-j}+\sum_i e^{-y_i(Z^p,W)})) $$
Cuya intención es hacer competir a las clases y penalizarlas.  

**Optimizador (Gradiente descendiente y/o Adam)**


In [40]:
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.SGD(learning_rate=0.25),metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_4 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_5 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_5 (Dense)              (None, 84)               

# Entrenamiento

Debido a la alta cantidad de datos, se realizará un batch para poder tener una muestra significativa de los valores

In [41]:
import time
import random
m=random.randint(1,1000)
print(m)
print(x_conv_p[0:m,:,:].shape,yp_oh[0:m,:].shape)
t1=time.time()
model.fit(x_conv_t,yt_oh,epochs=3,verbose=1,batch_size=64)
t2=time.time()
print('Tiempo de entrenamiento', t2-t1)

21
(21, 32, 32, 1) (21, 10)
Epoch 1/3
938/938 [==============================] - 45s 46ms/step - loss: 0.2407 - accuracy: 0.9277
Epoch 2/3
938/938 [==============================] - 41s 43ms/step - loss: 0.0754 - accuracy: 0.9767
Epoch 3/3
938/938 [==============================] - 40s 42ms/step - loss: 0.0507 - accuracy: 0.9842
Tiempo de entrenamiento 125.24375748634338


# Prueba (Test)

In [42]:
pred=model.predict(x_conv_p)
pred=np.argmax(pred,axis=1)
#label=np.argmax(yp_oh)
exactitud_test=0
for a in range(len(pred)):
    if pred[a]==Y1[a]:
        exactitud_test+=1
print('exactitud de la prueba= ',100*exactitud_test/len(pred),'%')

exactitud de la prueba=  98.34 %
